In [1]:
// The code was removed by DSX for sharing.

Waiting for a Spark session to start...

credentials = Map(apiKey -> UC2EdaFY_rERXSOZHgTciY1Prmqsnc4aGwpCzOSFun__, serviceId -> iam-ServiceId-d7c9135c-9c60-45fb-970a-c90d813805c4, endPoint -> https://s3-api.us-geo.objectstorage.service.networklayer.com, iamServiceEndpoint -> https://iam.ng.bluemix.net/oidc/token)
configurationName = os_45ef92514c2049f5ad540ea2a90e5504_configs
cos = com.ibm.ibmos2spark.CloudObjectStorage@21023a86


com.ibm.ibmos2spark.CloudObjectStorage@21023a86

In [2]:
val starttime = System.currentTimeMillis()
var test = sc.textFile(cos.url("bufantest-donotdelete-pr-tkuiqj2r7rommw", "video_small_testing_num.csv"))
val testheader = test.first()
test = test.filter(x=>x!=testheader)

Waiting for a Spark session to start...

starttime = 1528902157828
test = MapPartitionsRDD[2] at filter at <console>:38
testheader = userID,productID,rating
test = MapPartitionsRDD[2] at filter at <console>:38


MapPartitionsRDD[2] at filter at <console>:38

In [3]:
var train = sc.textFile(cos.url("bufantest-donotdelete-pr-tkuiqj2r7rommw", "video_small_num.csv"))
val trainheader = train.first()
train = train.filter(x=>x!=trainheader)

train = MapPartitionsRDD[5] at filter at <console>:37
trainheader = userID,productID, rating, timestamp
train = MapPartitionsRDD[5] at filter at <console>:37


MapPartitionsRDD[5] at filter at <console>:37

In [4]:
val test_ratings = test.map(_.split(',')).map(row=> (row(0).toInt, row(1).toInt, row(2).toDouble))
val train_ratings = train.map(_.split(',')).map(row => (row(0).toInt, row(1).toInt, row(2).toDouble))
val ratings = train_ratings.map(
            info => {
                Rating(info._1, info._2, info._3)
        })
val usersProducts = test_ratings.map (row => {(row._1, row._2)})


test_ratings = MapPartitionsRDD[7] at map at <console>:39
train_ratings = MapPartitionsRDD[9] at map at <console>:40
ratings = MapPartitionsRDD[10] at map at <console>:41
usersProducts = MapPartitionsRDD[11] at map at <console>:45


MapPartitionsRDD[11] at map at <console>:45

In [5]:
val rank = 5
val numIterations = 10
val lambda = 0.5
val seed = 1
val model = ALS.train(ratings, rank, numIterations, lambda, -1, seed)
//        val model = ALS.train(ratings, rank, numIterations,0.01)

var predictions = model.predict(usersProducts)
        .map { case Rating(user, product, rate) =>((user, product), rate)}


rank = 5
numIterations = 10
lambda = 0.5
seed = 1
model = org.apache.spark.mllib.recommendation.MatrixFactorizationModel@73360b7d
predictions = MapPartitionsRDD[227] at map at <console>:52


MapPartitionsRDD[227] at map at <console>:52

In [6]:
val min = predictions.map(_._2).min()
val max = predictions.map(_._2).max()
var result = predictions.map{case ((user, product), rate) =>
    ((user, product), 4.0 * (rate-min)/(max - min) + 1.0)

}

min = -3.7682779979313046
max = 5.182124623439929
result = MapPartitionsRDD[230] at map at <console>:53


MapPartitionsRDD[230] at map at <console>:53

In [ ]:
val ratesAndPreds = test_ratings.map { case (user, product, rate) =>
            ((user, product), rate)
        }.join(result)
val diff = ratesAndPreds.map{case ((user, product), (r1, r2)) =>
math.abs(r1 - r2)}
val RMSE = math.sqrt(diff.map(x=>{x * x}).mean())
var range1 = diff.filter{ case (diff) => diff>=0 && diff<1}.count()
var range2 = diff.filter{ case (diff) => diff>=1 && diff<2}.count()
var range3 = diff.filter{ case (diff) => diff>=2 && diff<3}.count()
var range4 = diff.filter{ case (diff) => diff>=3 && diff<4}.count()
var range5 = diff.filter{ case (diff) => diff>=4}.count()
println(">=0 and <1: "+range1)
println(">=1 and <2: "+range2)
println(">=2 and <3: "+range3)
println(">=3 and <4: "+range4)
println(">=4: "+range5)
println("RMSE: "+ RMSE)
println("Time: "+ ((System.currentTimeMillis()-starttime)/1000).toString +" sec")


>=0 and <1: 4845
>=1 and <2: 1944
>=2 and <3: 515
>=3 and <4: 396
>=4: 0
RMSE: 1.2947936470436476
Time: 76 sec
